In [1]:
import pandas as pd
import datetime
import calendar
import matplotlib.pyplot as plt
import numpy as np
import math
from layouts import prophet_forecast,arima_forecast,iterateclean,ets_forecast,min_mape
from flask import Flask
from flask import request
from flask import render_template
from flask import send_file
from werkzeug import secure_filename
from flask import flash,redirect,url_for,abort
import itertools

ModuleNotFoundError: No module named 'rpy2'

In [ ]:
import mysql.connector
from mysql.connector import Error
try:
    conn = mysql.connector.connect(host='127.0.0.1',
                                       port='3306',
                                       database='db',
                                       user='ankit',
                                       password='ankit123')
    if conn.is_connected():
        print('Connected to MySQL database')
    else:
        print("nope")
except Error as e:
    print(e)

In [19]:
app = Flask(__name__, template_folder = 'template')

In [20]:
@app.route('/', methods=['GET', 'POST'])
def index():
    masterfile=pd.read_excel(r"privatedata\ValidMasterCodes.xlsx")
    mastercodes=list(masterfile.MASTER.values)
    masterdata=pd.read_excel(r"privatedata\MasterData.xlsx")
    if request.method == 'GET':
        return render_template('index.html')
    elif request.method == 'POST':
        if request.form['action'] == 'Generate':
            inputmethod=request.form.get('InputMethod')
            model=request.form.get('Model')
            aggregator=request.form.get('AggregateBy')
            startdate=request.form.get('startdate')+'-01'
            enddate=request.form.get('enddate')+'-01'
            period=request.form.get('period')
            print(inputmethod)
            print(model)
            print(aggregator)
            print(startdate)
            print(enddate)
            print(period)
            ####################################    SQL    #######################################################
            if inputmethod.strip()=='sql' and aggregator.strip()=='mastercode':
                
                curr=conn.cursor()
                
                curr.execute("select * from shipments")
                quantity=curr.fetchall()
                data=pd.DataFrame(quantity,columns=['Group','Month','Total'])

                if model.strip()=='bestfit':
                    finaldf=pd.DataFrame()
                    mape=pd.DataFrame()
                    model=pd.DataFrame()
                    columnhead=[]

                    for master in mastercodes:
                        count=0
                        flag=''
                        cleandata,originaldata=iterateclean(data,master,startdate,enddate)
                        if len(originaldata)>12 and int(max(originaldata.index).strftime("%Y"))==2019:
                            try:
                                prophet_out,prophet_mape=prophet_forecast(cleandata,originaldata,int(period))
                                flag+='a'
                            except:
                                count+=1
                                pass
                            try:
                                arima_out,arima_mape=arima_forecast(cleandata,originaldata,int(period))
                                flag+='b'
                            except:
                                count+=1
                                pass
                            try:
                                ets_out,ets_mape=ets_forecast(cleandata,originaldata,int(period))
                                flag+='c'
                            except:
                                count+=1
                                pass
                        else:
                            continue
                        print(master)
                        if count<3:
                            if count==2:
                                if flag=='a':
                                    best=0
                                elif flag=='b':
                                    best=1
                                elif flag=='c':
                                    best=2
                            elif count==1:
                                if flag=='ab':
                                    best=min_mape(prophet_mape,arima_mape)
                                elif flag=='bc':
                                    best=min_mape(arima_mape,ets_mape)+1
                                elif flag=='ac':
                                    best=min_mape(prophet_mape,ets_mape)
                                    if best==1:
                                        best+=1
                            elif count==0:
                                best=min_mape(prophet_mape,arima_mape,ets_mape)
                            if best==0:
                                model[master]=['Prophet']
                                mape[master]=[prophet_mape]
                                try:
                                    finaldf=pd.concat([finaldf,prophet_out], ignore_index=True, axis=1)
                                    columnhead.append(master)
                                except:
                                    continue
                            elif best==1:
                                model[master]=['ARIMA']
                                mape[master]=[arima_mape]
                                try:
                                    finaldf=pd.concat([finaldf,arima_out], ignore_index=True, axis=1)
                                    columnhead.append(master)
                                except:
                                    continue
                            elif best==2:
                                model[master]=['ETS']
                                mape[master]=[ets_mape]
                                try:
                                    finaldf=pd.concat([finaldf,ets_out], ignore_index=True, axis=1)
                                    columnhead.append(master)
                                except:
                                    continue
                            else:
                                continue
                        else:
                            continue

                    finaldf.columns=columnhead
                    mape=mape.transpose()
                    model=model.transpose()
                    mape.columns=['Mape']
                    model.columns=['Model']
                    exportdf=finaldf.transpose()
                    exportdf["Model"]=model.Model
                    exportdf['Mape']=mape.Mape
                    exportdf.to_excel("forecastsheet.xlsx")

                    return send_file('forecastsheet.xlsx', attachment_filename='forecastsheet.xlsx')

                elif model.strip()=='prophet':
                    finaldf=pd.DataFrame()
                    mape=pd.DataFrame()
                    model=pd.DataFrame()
                    columnhead=[]

                    for master in mastercodes:
                        cleandata,originaldata=iterateclean(data,master,startdate,enddate)
                        if len(originaldata)>12 and int(max(originaldata.index).strftime("%Y"))==2019:
                            try:
                                prophet_out,prophet_mape=prophet_forecast(cleandata,originaldata,int(period))
                            except:
                                pass
                        else:
                            continue

                        try:
                            finaldf=pd.concat([finaldf,prophet_out], ignore_index=True, axis=1)
                            columnhead.append(master)
                        except:
                            continue
                        model[master]=['Prophet']
                        mape[master]=[prophet_mape]

                    finaldf.columns=columnhead
                    mape=mape.transpose()
                    model=model.transpose()
                    mape.columns=['Mape']
                    model.columns=['Model']
                    exportdf=finaldf.transpose()
                    exportdf["Model"]=model.Model
                    exportdf['Mape']=mape.Mape
                    exportdf.to_excel("forecastsheet.xlsx")

                    return send_file('forecastsheet.xlsx', attachment_filename='forecastsheet.xlsx')
                
                elif model.strip()=='arima':
                    finaldf=pd.DataFrame()
                    mape=pd.DataFrame()
                    model=pd.DataFrame()
                    columnhead=[]

                    for master in mastercodes:
                        cleandata,originaldata=iterateclean(data,master,startdate,enddate)
                        if len(originaldata)>12 and int(max(originaldata.index).strftime("%Y"))==2019:
                            try:
                                arima_out,arima_mape=arima_forecast(cleandata,originaldata,int(period))
                            except:
                                pass
                        else:
                            continue

                        try:
                            finaldf=pd.concat([finaldf,arima_out], ignore_index=True, axis=1)
                            columnhead.append(master)
                        except:
                            continue
                        model[master]=['ARIMA']
                        mape[master]=[arima_mape]

                    finaldf.columns=columnhead
                    mape=mape.transpose()
                    model=model.transpose()
                    mape.columns=['Mape']
                    model.columns=['Model']
                    exportdf=finaldf.transpose()
                    exportdf["Model"]=model.Model
                    exportdf['Mape']=mape.Mape
                    exportdf.to_excel("forecastsheet.xlsx")

                    return send_file('forecastsheet.xlsx', attachment_filename='forecastsheet.xlsx')                
        
        
                elif model.strip()=='ets':
                    finaldf=pd.DataFrame()
                    mape=pd.DataFrame()
                    model=pd.DataFrame()
                    columnhead=[]

                    for master in mastercodes:
                        cleandata,originaldata=iterateclean(data,master,startdate,enddate)
                        if len(originaldata)>12 and int(max(originaldata.index).strftime("%Y"))==2019:
                            try:
                                ets_out,ets_mape=ets_forecast(cleandata,originaldata,int(period))
                            except:
                                pass
                        else:
                            continue

                        try:
                            finaldf=pd.concat([finaldf,ets_out], ignore_index=True, axis=1)
                            columnhead.append(master)
                        except:
                            continue
                        model[master]=['ETS']
                        mape[master]=[ets_mape]

                    finaldf.columns=columnhead
                    mape=mape.transpose()
                    model=model.transpose()
                    mape.columns=['Mape']
                    model.columns=['Model']
                    exportdf=finaldf.transpose()
                    exportdf["Model"]=model.Model
                    exportdf['Mape']=mape.Mape
                    exportdf.to_excel("forecastsheet.xlsx")

                    return send_file('forecastsheet.xlsx', attachment_filename='forecastsheet.xlsx')

######################################################  CSV   ##############################################################
            
    
            if inputmethod.strip()=='csv' and aggregator.strip()=='mastercode':
                f = request.files['csvfile']
                f.save(secure_filename(f.filename))
                print(f.filename)
                if str(f.filename).endswith('.csv'):
                    data=pd.read_csv("%s"%(secure_filename(f.filename)),encoding='latin')
                    try:
                        data=data[['Group','Month','Total']]
                    except:
                        return redirect(url_for('index'))
                if str(f.filename).endswith('.xlsx') or str(f.filename).endswith('.xls'):
                    data=pd.read_excel("%s"%(secure_filename(f.filename)))
                    try:
                        data=data[['Group','Month','Total']]
                    except:
                        return redirect(url_for('index'))
                if model.strip()=='bestfit':
                    finaldf=pd.DataFrame()
                    mape=pd.DataFrame()
                    model=pd.DataFrame()
                    columnhead=[]

                    for master in mastercodes:
                        count=0
                        flag=''
                        cleandata,originaldata=iterateclean(data,'Group',master,startdate,enddate)
                        if len(originaldata)>12 and int(max(originaldata.index).strftime("%Y"))==2019:
                            try:
                                prophet_out,prophet_mape=prophet_forecast(cleandata,originaldata,int(period))
                                flag+='a'
                            except:
                                count+=1
                                pass
                            try:
                                arima_out,arima_mape=arima_forecast(cleandata,originaldata,int(period))
                                flag+='b'
                            except:
                                count+=1
                                pass
                            try:
                                ets_out,ets_mape=ets_forecast(cleandata,originaldata,int(period))
                                flag+='c'
                            except:
                                count+=1
                                pass
                        else:
                            continue
                        print(master)
                        if count<3:
                            if count==2:
                                if flag=='a':
                                    best=0
                                elif flag=='b':
                                    best=1
                                elif flag=='c':
                                    best=2
                            elif count==1:
                                if flag=='ab':
                                    best=min_mape(prophet_mape,arima_mape)
                                elif flag=='bc':
                                    best=min_mape(arima_mape,ets_mape)+1
                                elif flag=='ac':
                                    best=min_mape(prophet_mape,ets_mape)
                                    if best==1:
                                        best+=1
                            elif count==0:
                                best=min_mape(prophet_mape,arima_mape,ets_mape)
                            if best==0:
                                model[master]=['Prophet']
                                mape[master]=[prophet_mape]
                                try:
                                    finaldf=pd.concat([finaldf,prophet_out], ignore_index=True, axis=1)
                                    columnhead.append(master)
                                except:
                                    continue
                            elif best==1:
                                model[master]=['ARIMA']
                                mape[master]=[arima_mape]
                                try:
                                    finaldf=pd.concat([finaldf,arima_out], ignore_index=True, axis=1)
                                    columnhead.append(master)
                                except:
                                    continue
                            elif best==2:
                                model[master]=['ETS']
                                mape[master]=[ets_mape]
                                try:
                                    finaldf=pd.concat([finaldf,ets_out], ignore_index=True, axis=1)
                                    columnhead.append(master)
                                except:
                                    continue
                            else:
                                continue
                        else:
                            continue

                    finaldf.columns=columnhead
                    mape=mape.transpose()
                    model=model.transpose()
                    mape.columns=['Mape']
                    model.columns=['Model']
                    exportdf=finaldf.transpose()
                    exportdf["Model"]=model.Model
                    exportdf['Mape']=mape.Mape
                    exportdf.to_excel("forecastsheet.xlsx")

                    return send_file('forecastsheet.xlsx', attachment_filename='forecastsheet.xlsx')

                elif model.strip()=='prophet':
                    finaldf=pd.DataFrame()
                    mape=pd.DataFrame()
                    model=pd.DataFrame()
                    columnhead=[]

                    for master in mastercodes:
                        cleandata,originaldata=iterateclean(data,'Group',master,startdate,enddate)
                        if len(originaldata)>12 and int(max(originaldata.index).strftime("%Y"))==2019:
                            try:
                                prophet_out,prophet_mape=prophet_forecast(cleandata,originaldata,int(period))
                            except:
                                pass
                        else:
                            continue

                        try:
                            finaldf=pd.concat([finaldf,prophet_out], ignore_index=True, axis=1)
                            columnhead.append(master)
                        except:
                            continue
                        model[master]=['Prophet']
                        mape[master]=[prophet_mape]

                    finaldf.columns=columnhead
                    mape=mape.transpose()
                    model=model.transpose()
                    mape.columns=['Mape']
                    model.columns=['Model']
                    exportdf=finaldf.transpose()
                    exportdf["Model"]=model.Model
                    exportdf['Mape']=mape.Mape
                    exportdf.to_excel("forecastsheet.xlsx")

                    return send_file('forecastsheet.xlsx', attachment_filename='forecastsheet.xlsx')
                
                elif model.strip()=='arima':
                    finaldf=pd.DataFrame()
                    mape=pd.DataFrame()
                    model=pd.DataFrame()
                    columnhead=[]

                    for master in mastercodes:
                        cleandata,originaldata=iterateclean(data,'Group',master,startdate,enddate)
                        if len(originaldata)>12 and int(max(originaldata.index).strftime("%Y"))==2019:
                            try:
                                arima_out,arima_mape=arima_forecast(cleandata,originaldata,int(period))
                            except:
                                pass
                        else:
                            continue

                        try:
                            finaldf=pd.concat([finaldf,arima_out], ignore_index=True, axis=1)
                            columnhead.append(master)
                        except:
                            continue
                        model[master]=['ARIMA']
                        mape[master]=[arima_mape]

                    finaldf.columns=columnhead
                    mape=mape.transpose()
                    model=model.transpose()
                    mape.columns=['Mape']
                    model.columns=['Model']
                    exportdf=finaldf.transpose()
                    exportdf["Model"]=model.Model
                    exportdf['Mape']=mape.Mape
                    exportdf.to_excel("forecastsheet.xlsx")

                    return send_file('forecastsheet.xlsx', attachment_filename='forecastsheet.xlsx')                
        
        
                elif model.strip()=='ets':
                    finaldf=pd.DataFrame()
                    mape=pd.DataFrame()
                    model=pd.DataFrame()
                    columnhead=[]

                    for master in mastercodes:
                        cleandata,originaldata=iterateclean(data,'Group',master,startdate,enddate)
                        if len(originaldata)>12 and int(max(originaldata.index).strftime("%Y"))==2019:
                            try:
                                ets_out,ets_mape=ets_forecast(cleandata,originaldata,int(period))
                            except:
                                pass
                        else:
                            continue

                        try:
                            finaldf=pd.concat([finaldf,ets_out], ignore_index=True, axis=1)
                            columnhead.append(master)
                        except:
                            continue
                        model[master]=['ETS']
                        mape[master]=[ets_mape]

                    finaldf.columns=columnhead
                    mape=mape.transpose()
                    model=model.transpose()
                    mape.columns=['Mape']
                    model.columns=['Model']
                    exportdf=finaldf.transpose()
                    exportdf["Model"]=model.Model
                    exportdf['Mape']=mape.Mape
                    exportdf.to_excel("forecastsheet.xlsx")

                    return send_file('forecastsheet.xlsx', attachment_filename='forecastsheet.xlsx')
       
    ##################################################   aggregate by brand ##########################################
            if inputmethod.strip()=='csv' and aggregator.strip()=='brand':
                f = request.files['csvfile']
                f.save(secure_filename(f.filename))
                print(f.filename)
                if str(f.filename).endswith('.csv'):
                    data=pd.read_csv("%s"%(secure_filename(f.filename)),encoding='latin')
                    try:
                        data=data[['Group','Month','Total']]
                    except:
                        return redirect(url_for('index'))
                if str(f.filename).endswith('.xlsx') or str(f.filename).endswith('.xls'):
                    data=pd.read_excel("%s"%(secure_filename(f.filename)))
                    try:
                        data=data[['Group','Month','Total']]
                    except:
                        return redirect(url_for('index'))
                    
                joined=pd.merge(data, masterdata, left_on='Group',right_on='Cat Cd10 Description', how='left')   
                bybrand=joined.groupby(['Brand Description','Month'])[['Total']].sum()    
                bybrand.reset_index(inplace=True)
                brands=list(bybrand['Brand Description'].unique())
                    
                if model.strip()=='bestfit':
                    finaldf=pd.DataFrame()
                    mape=pd.DataFrame()
                    model=pd.DataFrame()
                    columnhead=[]

                    for master in brands:
                        count=0
                        flag=''
                        cleandata,originaldata=iterateclean(bybrand,'Brand Description',master,startdate,enddate)
                        if len(originaldata)>12 and int(max(originaldata.index).strftime("%Y"))==2019:
                            try:
                                prophet_out,prophet_mape=prophet_forecast(cleandata,originaldata,int(period))
                                flag+='a'
                            except:
                                count+=1
                                pass
                            try:
                                arima_out,arima_mape=arima_forecast(cleandata,originaldata,int(period))
                                flag+='b'
                            except:
                                count+=1
                                pass
                            try:
                                ets_out,ets_mape=ets_forecast(cleandata,originaldata,int(period))
                                flag+='c'
                            except:
                                count+=1
                                pass
                        else:
                            continue
                        print(master)
                        if count<3:
                            if count==2:
                                if flag=='a':
                                    best=0
                                elif flag=='b':
                                    best=1
                                elif flag=='c':
                                    best=2
                            elif count==1:
                                if flag=='ab':
                                    best=min_mape(prophet_mape,arima_mape)
                                elif flag=='bc':
                                    best=min_mape(arima_mape,ets_mape)+1
                                elif flag=='ac':
                                    best=min_mape(prophet_mape,ets_mape)
                                    if best==1:
                                        best+=1
                            elif count==0:
                                best=min_mape(prophet_mape,arima_mape,ets_mape)
                            if best==0:
                                model[master]=['Prophet']
                                mape[master]=[prophet_mape]
                                try:
                                    finaldf=pd.concat([finaldf,prophet_out], ignore_index=True, axis=1)
                                    columnhead.append(master)
                                except:
                                    continue
                            elif best==1:
                                model[master]=['ARIMA']
                                mape[master]=[arima_mape]
                                try:
                                    finaldf=pd.concat([finaldf,arima_out], ignore_index=True, axis=1)
                                    columnhead.append(master)
                                except:
                                    continue
                            elif best==2:
                                model[master]=['ETS']
                                mape[master]=[ets_mape]
                                try:
                                    finaldf=pd.concat([finaldf,ets_out], ignore_index=True, axis=1)
                                    columnhead.append(master)
                                except:
                                    continue
                            else:
                                continue
                        else:
                            continue

                    finaldf.columns=columnhead
                    mape=mape.transpose()
                    model=model.transpose()
                    mape.columns=['Mape']
                    model.columns=['Model']
                    exportdf=finaldf.transpose()
                    exportdf["Model"]=model.Model
                    exportdf['Mape']=mape.Mape
                    exportdf.to_excel("forecastsheet.xlsx")

                    return send_file('forecastsheet.xlsx', attachment_filename='forecastsheet.xlsx')

                elif model.strip()=='prophet':
                    finaldf=pd.DataFrame()
                    mape=pd.DataFrame()
                    model=pd.DataFrame()
                    columnhead=[]

                    for master in brands:
                        cleandata,originaldata=iterateclean(bybrand,'Brand Description',master,startdate,enddate)
                        if len(originaldata)>12 and int(max(originaldata.index).strftime("%Y"))==2019:
                            try:
                                prophet_out,prophet_mape=prophet_forecast(cleandata,originaldata,int(period))
                            except:
                                pass
                        else:
                            continue

                        try:
                            finaldf=pd.concat([finaldf,prophet_out], ignore_index=True, axis=1)
                            columnhead.append(master)
                        except:
                            continue
                        model[master]=['Prophet']
                        mape[master]=[prophet_mape]

                    finaldf.columns=columnhead
                    mape=mape.transpose()
                    model=model.transpose()
                    mape.columns=['Mape']
                    model.columns=['Model']
                    exportdf=finaldf.transpose()
                    exportdf["Model"]=model.Model
                    exportdf['Mape']=mape.Mape
                    exportdf.to_excel("forecastsheet.xlsx")

                    return send_file('forecastsheet.xlsx', attachment_filename='forecastsheet.xlsx')
                
                elif model.strip()=='arima':
                    finaldf=pd.DataFrame()
                    mape=pd.DataFrame()
                    model=pd.DataFrame()
                    columnhead=[]

                    for master in brands:
                        cleandata,originaldata=iterateclean(bybrand,'Brand Description',master,startdate,enddate)
                        if len(originaldata)>12 and int(max(originaldata.index).strftime("%Y"))==2019:
                            try:
                                arima_out,arima_mape=arima_forecast(cleandata,originaldata,int(period))
                            except:
                                pass
                        else:
                            continue

                        try:
                            finaldf=pd.concat([finaldf,arima_out], ignore_index=True, axis=1)
                            columnhead.append(master)
                        except:
                            continue
                        model[master]=['ARIMA']
                        mape[master]=[arima_mape]

                    finaldf.columns=columnhead
                    mape=mape.transpose()
                    model=model.transpose()
                    mape.columns=['Mape']
                    model.columns=['Model']
                    exportdf=finaldf.transpose()
                    exportdf["Model"]=model.Model
                    exportdf['Mape']=mape.Mape
                    exportdf.to_excel("forecastsheet.xlsx")

                    return send_file('forecastsheet.xlsx', attachment_filename='forecastsheet.xlsx')                
        
        
                elif model.strip()=='ets':
                    finaldf=pd.DataFrame()
                    mape=pd.DataFrame()
                    model=pd.DataFrame()
                    columnhead=[]

                    for master in brands:
                        cleandata,originaldata=iterateclean(bybrand,'Brand Description',master,startdate,enddate)
                        if len(originaldata)>12 and int(max(originaldata.index).strftime("%Y"))==2019:
                            try:
                                ets_out,ets_mape=ets_forecast(cleandata,originaldata,int(period))
                            except:
                                pass
                        else:
                            continue

                        try:
                            finaldf=pd.concat([finaldf,ets_out], ignore_index=True, axis=1)
                            columnhead.append(master)
                        except:
                            continue
                        model[master]=['ETS']
                        mape[master]=[ets_mape]

                    finaldf.columns=columnhead
                    mape=mape.transpose()
                    model=model.transpose()
                    mape.columns=['Mape']
                    model.columns=['Model']
                    exportdf=finaldf.transpose()
                    exportdf["Model"]=model.Model
                    exportdf['Mape']=mape.Mape
                    exportdf.to_excel("forecastsheet.xlsx")

                    return send_file('forecastsheet.xlsx', attachment_filename='forecastsheet.xlsx')
                
     ##################################################   aggregate by size ##########################################
            if inputmethod.strip()=='csv' and aggregator.strip()=='size':
                f = request.files['csvfile']
                f.save(secure_filename(f.filename))
                print(f.filename)
                if str(f.filename).endswith('.csv'):
                    data=pd.read_csv("%s"%(secure_filename(f.filename)),encoding='latin')
                    try:
                        data=data[['Group','Month','Total']]
                    except:
                        return redirect(url_for('index'))
                if str(f.filename).endswith('.xlsx') or str(f.filename).endswith('.xls'):
                    data=pd.read_excel("%s"%(secure_filename(f.filename)))
                    try:
                        data=data[['Group','Month','Total']]
                    except:
                        return redirect(url_for('index'))
                    
                joined=pd.merge(data, masterdata, left_on='Group',right_on='Cat Cd10 Description', how='left')   
                bysize=joined.groupby(['Size Description','Month'])[['Total']].sum()    
                bysize.reset_index(inplace=True)
                sizes=list(bysize['Size Description'].unique())
                    
                if model.strip()=='bestfit':
                    finaldf=pd.DataFrame()
                    mape=pd.DataFrame()
                    model=pd.DataFrame()
                    columnhead=[]

                    for master in sizes:
                        count=0
                        flag=''
                        cleandata,originaldata=iterateclean(bysize,'Size Description',master,startdate,enddate)
                        if len(originaldata)>12 and int(max(originaldata.index).strftime("%Y"))==2019:
                            try:
                                prophet_out,prophet_mape=prophet_forecast(cleandata,originaldata,int(period))
                                flag+='a'
                            except:
                                count+=1
                                pass
                            try:
                                arima_out,arima_mape=arima_forecast(cleandata,originaldata,int(period))
                                flag+='b'
                            except:
                                count+=1
                                pass
                            try:
                                ets_out,ets_mape=ets_forecast(cleandata,originaldata,int(period))
                                flag+='c'
                            except:
                                count+=1
                                pass
                        else:
                            continue
                        print(master)
                        if count<3:
                            if count==2:
                                if flag=='a':
                                    best=0
                                elif flag=='b':
                                    best=1
                                elif flag=='c':
                                    best=2
                            elif count==1:
                                if flag=='ab':
                                    best=min_mape(prophet_mape,arima_mape)
                                elif flag=='bc':
                                    best=min_mape(arima_mape,ets_mape)+1
                                elif flag=='ac':
                                    best=min_mape(prophet_mape,ets_mape)
                                    if best==1:
                                        best+=1
                            elif count==0:
                                best=min_mape(prophet_mape,arima_mape,ets_mape)
                            if best==0:
                                model[master]=['Prophet']
                                mape[master]=[prophet_mape]
                                try:
                                    finaldf=pd.concat([finaldf,prophet_out], ignore_index=True, axis=1)
                                    columnhead.append(master)
                                except:
                                    continue
                            elif best==1:
                                model[master]=['ARIMA']
                                mape[master]=[arima_mape]
                                try:
                                    finaldf=pd.concat([finaldf,arima_out], ignore_index=True, axis=1)
                                    columnhead.append(master)
                                except:
                                    continue
                            elif best==2:
                                model[master]=['ETS']
                                mape[master]=[ets_mape]
                                try:
                                    finaldf=pd.concat([finaldf,ets_out], ignore_index=True, axis=1)
                                    columnhead.append(master)
                                except:
                                    continue
                            else:
                                continue
                        else:
                            continue

                    finaldf.columns=columnhead
                    mape=mape.transpose()
                    model=model.transpose()
                    mape.columns=['Mape']
                    model.columns=['Model']
                    exportdf=finaldf.transpose()
                    exportdf["Model"]=model.Model
                    exportdf['Mape']=mape.Mape
                    exportdf.to_excel("forecastsheet.xlsx")

                    return send_file('forecastsheet.xlsx', attachment_filename='forecastsheet.xlsx')

                elif model.strip()=='prophet':
                    finaldf=pd.DataFrame()
                    mape=pd.DataFrame()
                    model=pd.DataFrame()
                    columnhead=[]

                    for master in sizes:
                        cleandata,originaldata=iterateclean(bysize,'Size Description',master,startdate,enddate)
                        if len(originaldata)>12 and int(max(originaldata.index).strftime("%Y"))==2019:
                            try:
                                prophet_out,prophet_mape=prophet_forecast(cleandata,originaldata,int(period))
                            except:
                                pass
                        else:
                            continue

                        try:
                            finaldf=pd.concat([finaldf,prophet_out], ignore_index=True, axis=1)
                            columnhead.append(master)
                        except:
                            continue
                        model[master]=['Prophet']
                        mape[master]=[prophet_mape]

                    finaldf.columns=columnhead
                    mape=mape.transpose()
                    model=model.transpose()
                    mape.columns=['Mape']
                    model.columns=['Model']
                    exportdf=finaldf.transpose()
                    exportdf["Model"]=model.Model
                    exportdf['Mape']=mape.Mape
                    exportdf.to_excel("forecastsheet.xlsx")

                    return send_file('forecastsheet.xlsx', attachment_filename='forecastsheet.xlsx')
                
                elif model.strip()=='arima':
                    finaldf=pd.DataFrame()
                    mape=pd.DataFrame()
                    model=pd.DataFrame()
                    columnhead=[]

                    for master in sizes:
                        cleandata,originaldata=iterateclean(bysize,'Size Description',master,startdate,enddate)
                        if len(originaldata)>12 and int(max(originaldata.index).strftime("%Y"))==2019:
                            try:
                                arima_out,arima_mape=arima_forecast(cleandata,originaldata,int(period))
                            except:
                                pass
                        else:
                            continue

                        try:
                            finaldf=pd.concat([finaldf,arima_out], ignore_index=True, axis=1)
                            columnhead.append(master)
                        except:
                            continue
                        model[master]=['ARIMA']
                        mape[master]=[arima_mape]

                    finaldf.columns=columnhead
                    mape=mape.transpose()
                    model=model.transpose()
                    mape.columns=['Mape']
                    model.columns=['Model']
                    exportdf=finaldf.transpose()
                    exportdf["Model"]=model.Model
                    exportdf['Mape']=mape.Mape
                    exportdf.to_excel("forecastsheet.xlsx")

                    return send_file('forecastsheet.xlsx', attachment_filename='forecastsheet.xlsx')                
        
        
                elif model.strip()=='ets':
                    finaldf=pd.DataFrame()
                    mape=pd.DataFrame()
                    model=pd.DataFrame()
                    columnhead=[]

                    for master in sizes:
                        cleandata,originaldata=iterateclean(bysize,'Size Description',master,startdate,enddate)
                        if len(originaldata)>12 and int(max(originaldata.index).strftime("%Y"))==2019:
                            try:
                                ets_out,ets_mape=ets_forecast(cleandata,originaldata,int(period))
                            except:
                                pass
                        else:
                            continue

                        try:
                            finaldf=pd.concat([finaldf,ets_out], ignore_index=True, axis=1)
                            columnhead.append(master)
                        except:
                            continue
                        model[master]=['ETS']
                        mape[master]=[ets_mape]

                    finaldf.columns=columnhead
                    mape=mape.transpose()
                    model=model.transpose()
                    mape.columns=['Mape']
                    model.columns=['Model']
                    exportdf=finaldf.transpose()
                    exportdf["Model"]=model.Model
                    exportdf['Mape']=mape.Mape
                    exportdf.to_excel("forecastsheet.xlsx")

                    return send_file('forecastsheet.xlsx', attachment_filename='forecastsheet.xlsx')

     ##################################################   aggregate by varietal ##########################################
            if inputmethod.strip()=='csv' and aggregator.strip()=='varietal':
                f = request.files['csvfile']
                f.save(secure_filename(f.filename))
                print(f.filename)
                if str(f.filename).endswith('.csv'):
                    data=pd.read_csv("%s"%(secure_filename(f.filename)),encoding='latin')
                    try:
                        data=data[['Group','Month','Total']]
                    except:
                        return redirect(url_for('index'))
                if str(f.filename).endswith('.xlsx') or str(f.filename).endswith('.xls'):
                    data=pd.read_excel("%s"%(secure_filename(f.filename)))
                    try:
                        data=data[['Group','Month','Total']]
                    except:
                        return redirect(url_for('index'))
                    
                joined=pd.merge(data, masterdata, left_on='Group',right_on='Cat Cd10 Description', how='left')   
                byvarietal=joined.groupby(['Varietal/ Flavor Description','Month'])[['Total']].sum()    
                byvarietal.reset_index(inplace=True)
                varietals=list(byvarietal['Varietal/ Flavor Description'].unique())
                    
                if model.strip()=='bestfit':
                    finaldf=pd.DataFrame()
                    mape=pd.DataFrame()
                    model=pd.DataFrame()
                    columnhead=[]

                    for master in varietals:
                        count=0
                        flag=''
                        cleandata,originaldata=iterateclean(byvarietal,'Varietal/ Flavor Description',master,startdate,enddate)
                        if len(originaldata)>12 and int(max(originaldata.index).strftime("%Y"))==2019:
                            try:
                                prophet_out,prophet_mape=prophet_forecast(cleandata,originaldata,int(period))
                                flag+='a'
                            except:
                                count+=1
                                pass
                            try:
                                arima_out,arima_mape=arima_forecast(cleandata,originaldata,int(period))
                                flag+='b'
                            except:
                                count+=1
                                pass
                            try:
                                ets_out,ets_mape=ets_forecast(cleandata,originaldata,int(period))
                                flag+='c'
                            except:
                                count+=1
                                pass
                        else:
                            continue
                        print(master)
                        if count<3:
                            if count==2:
                                if flag=='a':
                                    best=0
                                elif flag=='b':
                                    best=1
                                elif flag=='c':
                                    best=2
                            elif count==1:
                                if flag=='ab':
                                    best=min_mape(prophet_mape,arima_mape)
                                elif flag=='bc':
                                    best=min_mape(arima_mape,ets_mape)+1
                                elif flag=='ac':
                                    best=min_mape(prophet_mape,ets_mape)
                                    if best==1:
                                        best+=1
                            elif count==0:
                                best=min_mape(prophet_mape,arima_mape,ets_mape)
                            if best==0:
                                model[master]=['Prophet']
                                mape[master]=[prophet_mape]
                                try:
                                    finaldf=pd.concat([finaldf,prophet_out], ignore_index=True, axis=1)
                                    columnhead.append(master)
                                except:
                                    continue
                            elif best==1:
                                model[master]=['ARIMA']
                                mape[master]=[arima_mape]
                                try:
                                    finaldf=pd.concat([finaldf,arima_out], ignore_index=True, axis=1)
                                    columnhead.append(master)
                                except:
                                    continue
                            elif best==2:
                                model[master]=['ETS']
                                mape[master]=[ets_mape]
                                try:
                                    finaldf=pd.concat([finaldf,ets_out], ignore_index=True, axis=1)
                                    columnhead.append(master)
                                except:
                                    continue
                            else:
                                continue
                        else:
                            continue

                    finaldf.columns=columnhead
                    mape=mape.transpose()
                    model=model.transpose()
                    mape.columns=['Mape']
                    model.columns=['Model']
                    exportdf=finaldf.transpose()
                    exportdf["Model"]=model.Model
                    exportdf['Mape']=mape.Mape
                    exportdf.to_excel("forecastsheet.xlsx")

                    return send_file('forecastsheet.xlsx', attachment_filename='forecastsheet.xlsx')

                elif model.strip()=='prophet':
                    finaldf=pd.DataFrame()
                    mape=pd.DataFrame()
                    model=pd.DataFrame()
                    columnhead=[]

                    for master in varietals:
                        cleandata,originaldata=iterateclean(byvarietal,'Varietal/ Flavor Description',master,startdate,enddate)
                        if len(originaldata)>12 and int(max(originaldata.index).strftime("%Y"))==2019:
                            try:
                                prophet_out,prophet_mape=prophet_forecast(cleandata,originaldata,int(period))
                            except:
                                pass
                        else:
                            continue

                        try:
                            finaldf=pd.concat([finaldf,prophet_out], ignore_index=True, axis=1)
                            columnhead.append(master)
                        except:
                            continue
                        model[master]=['Prophet']
                        mape[master]=[prophet_mape]

                    finaldf.columns=columnhead
                    mape=mape.transpose()
                    model=model.transpose()
                    mape.columns=['Mape']
                    model.columns=['Model']
                    exportdf=finaldf.transpose()
                    exportdf["Model"]=model.Model
                    exportdf['Mape']=mape.Mape
                    exportdf.to_excel("forecastsheet.xlsx")

                    return send_file('forecastsheet.xlsx', attachment_filename='forecastsheet.xlsx')
                
                elif model.strip()=='arima':
                    finaldf=pd.DataFrame()
                    mape=pd.DataFrame()
                    model=pd.DataFrame()
                    columnhead=[]

                    for master in varietals:
                        cleandata,originaldata=iterateclean(byvarietal,'Varietal/ Flavor Description',master,startdate,enddate)
                        if len(originaldata)>12 and int(max(originaldata.index).strftime("%Y"))==2019:
                            try:
                                arima_out,arima_mape=arima_forecast(cleandata,originaldata,int(period))
                            except:
                                pass
                        else:
                            continue

                        try:
                            finaldf=pd.concat([finaldf,arima_out], ignore_index=True, axis=1)
                            columnhead.append(master)
                        except:
                            continue
                        model[master]=['ARIMA']
                        mape[master]=[arima_mape]

                    finaldf.columns=columnhead
                    mape=mape.transpose()
                    model=model.transpose()
                    mape.columns=['Mape']
                    model.columns=['Model']
                    exportdf=finaldf.transpose()
                    exportdf["Model"]=model.Model
                    exportdf['Mape']=mape.Mape
                    exportdf.to_excel("forecastsheet.xlsx")

                    return send_file('forecastsheet.xlsx', attachment_filename='forecastsheet.xlsx')                
        
        
                elif model.strip()=='ets':
                    finaldf=pd.DataFrame()
                    mape=pd.DataFrame()
                    model=pd.DataFrame()
                    columnhead=[]

                    for master in varietals:
                        cleandata,originaldata=iterateclean(byvarietal,'Varietal/ Flavor Description',master,startdate,enddate)
                        if len(originaldata)>12 and int(max(originaldata.index).strftime("%Y"))==2019:
                            try:
                                ets_out,ets_mape=ets_forecast(cleandata,originaldata,int(period))
                            except:
                                pass
                        else:
                            continue

                        try:
                            finaldf=pd.concat([finaldf,ets_out], ignore_index=True, axis=1)
                            columnhead.append(master)
                        except:
                            continue
                        model[master]=['ETS']
                        mape[master]=[ets_mape]

                    finaldf.columns=columnhead
                    mape=mape.transpose()
                    model=model.transpose()
                    mape.columns=['Mape']
                    model.columns=['Model']
                    exportdf=finaldf.transpose()
                    exportdf["Model"]=model.Model
                    exportdf['Mape']=mape.Mape
                    exportdf.to_excel("forecastsheet.xlsx")

                    return send_file('forecastsheet.xlsx', attachment_filename='forecastsheet.xlsx')

                
     ##################################################   aggregate by brand+size ##########################################
            if inputmethod.strip()=='csv' and aggregator.strip()=='varietal':
                f = request.files['csvfile']
                f.save(secure_filename(f.filename))
                print(f.filename)
                if str(f.filename).endswith('.csv'):
                    data=pd.read_csv("%s"%(secure_filename(f.filename)),encoding='latin')
                    try:
                        data=data[['Group','Month','Total']]
                    except:
                        return redirect(url_for('index'))
                if str(f.filename).endswith('.xlsx') or str(f.filename).endswith('.xls'):
                    data=pd.read_excel("%s"%(secure_filename(f.filename)))
                    try:
                        data=data[['Group','Month','Total']]
                    except:
                        return redirect(url_for('index'))
                    
                joined=pd.merge(data, masterdata, left_on='Group',right_on='Cat Cd10 Description', how='left')   
                bybrandsize=joined.groupby(['Brand Description','Size Description','Month'])[['Total']].sum()     
                bybrandsize.reset_index(inplace=True)
                bybrandsize['Brand Description']=bybrandsize['Brand Description']+' -'+bybrandsize['Size Description']
                bybrandsize=bybrandsize.drop(['Size Description'],axis=1)
                brandsizes=list(bybrandsize['Brand Description'].unique())
                    
                if model.strip()=='bestfit':
                    finaldf=pd.DataFrame()
                    mape=pd.DataFrame()
                    model=pd.DataFrame()
                    columnhead=[]

                    for master in brandsizes:
                        count=0
                        flag=''
                        cleandata,originaldata=iterateclean(bybrandsize,'Brand Description',master,startdate,enddate)
                        if len(originaldata)>12 and int(max(originaldata.index).strftime("%Y"))==2019:
                            try:
                                prophet_out,prophet_mape=prophet_forecast(cleandata,originaldata,int(period))
                                flag+='a'
                            except:
                                count+=1
                                pass
                            try:
                                arima_out,arima_mape=arima_forecast(cleandata,originaldata,int(period))
                                flag+='b'
                            except:
                                count+=1
                                pass
                            try:
                                ets_out,ets_mape=ets_forecast(cleandata,originaldata,int(period))
                                flag+='c'
                            except:
                                count+=1
                                pass
                        else:
                            continue
                        print(master)
                        if count<3:
                            if count==2:
                                if flag=='a':
                                    best=0
                                elif flag=='b':
                                    best=1
                                elif flag=='c':
                                    best=2
                            elif count==1:
                                if flag=='ab':
                                    best=min_mape(prophet_mape,arima_mape)
                                elif flag=='bc':
                                    best=min_mape(arima_mape,ets_mape)+1
                                elif flag=='ac':
                                    best=min_mape(prophet_mape,ets_mape)
                                    if best==1:
                                        best+=1
                            elif count==0:
                                best=min_mape(prophet_mape,arima_mape,ets_mape)
                            if best==0:
                                model[master]=['Prophet']
                                mape[master]=[prophet_mape]
                                try:
                                    finaldf=pd.concat([finaldf,prophet_out], ignore_index=True, axis=1)
                                    columnhead.append(master)
                                except:
                                    continue
                            elif best==1:
                                model[master]=['ARIMA']
                                mape[master]=[arima_mape]
                                try:
                                    finaldf=pd.concat([finaldf,arima_out], ignore_index=True, axis=1)
                                    columnhead.append(master)
                                except:
                                    continue
                            elif best==2:
                                model[master]=['ETS']
                                mape[master]=[ets_mape]
                                try:
                                    finaldf=pd.concat([finaldf,ets_out], ignore_index=True, axis=1)
                                    columnhead.append(master)
                                except:
                                    continue
                            else:
                                continue
                        else:
                            continue

                    finaldf.columns=columnhead
                    mape=mape.transpose()
                    model=model.transpose()
                    mape.columns=['Mape']
                    model.columns=['Model']
                    exportdf=finaldf.transpose()
                    exportdf["Model"]=model.Model
                    exportdf['Mape']=mape.Mape
                    exportdf.to_excel("forecastsheet.xlsx")

                    return send_file('forecastsheet.xlsx', attachment_filename='forecastsheet.xlsx')

                elif model.strip()=='prophet':
                    finaldf=pd.DataFrame()
                    mape=pd.DataFrame()
                    model=pd.DataFrame()
                    columnhead=[]

                    for master in brandsizes:
                        cleandata,originaldata=iterateclean(bybrandsize,'Brand Description',master,startdate,enddate)
                        if len(originaldata)>12 and int(max(originaldata.index).strftime("%Y"))==2019:
                            try:
                                prophet_out,prophet_mape=prophet_forecast(cleandata,originaldata,int(period))
                            except:
                                pass
                        else:
                            continue

                        try:
                            finaldf=pd.concat([finaldf,prophet_out], ignore_index=True, axis=1)
                            columnhead.append(master)
                        except:
                            continue
                        model[master]=['Prophet']
                        mape[master]=[prophet_mape]

                    finaldf.columns=columnhead
                    mape=mape.transpose()
                    model=model.transpose()
                    mape.columns=['Mape']
                    model.columns=['Model']
                    exportdf=finaldf.transpose()
                    exportdf["Model"]=model.Model
                    exportdf['Mape']=mape.Mape
                    exportdf.to_excel("forecastsheet.xlsx")

                    return send_file('forecastsheet.xlsx', attachment_filename='forecastsheet.xlsx')
                
                elif model.strip()=='arima':
                    finaldf=pd.DataFrame()
                    mape=pd.DataFrame()
                    model=pd.DataFrame()
                    columnhead=[]

                    for master in brandsizes:
                        cleandata,originaldata=iterateclean(bybrandsize,'Brand Description',master,startdate,enddate)
                        if len(originaldata)>12 and int(max(originaldata.index).strftime("%Y"))==2019:
                            try:
                                arima_out,arima_mape=arima_forecast(cleandata,originaldata,int(period))
                            except:
                                pass
                        else:
                            continue

                        try:
                            finaldf=pd.concat([finaldf,arima_out], ignore_index=True, axis=1)
                            columnhead.append(master)
                        except:
                            continue
                        model[master]=['ARIMA']
                        mape[master]=[arima_mape]

                    finaldf.columns=columnhead
                    mape=mape.transpose()
                    model=model.transpose()
                    mape.columns=['Mape']
                    model.columns=['Model']
                    exportdf=finaldf.transpose()
                    exportdf["Model"]=model.Model
                    exportdf['Mape']=mape.Mape
                    exportdf.to_excel("forecastsheet.xlsx")

                    return send_file('forecastsheet.xlsx', attachment_filename='forecastsheet.xlsx')                
        
        
                elif model.strip()=='ets':
                    finaldf=pd.DataFrame()
                    mape=pd.DataFrame()
                    model=pd.DataFrame()
                    columnhead=[]

                    for master in brandsizes:
                        cleandata,originaldata=iterateclean(bybrandsize,'Brand Description',master,startdate,enddate)
                        if len(originaldata)>12 and int(max(originaldata.index).strftime("%Y"))==2019:
                            try:
                                ets_out,ets_mape=ets_forecast(cleandata,originaldata,int(period))
                            except:
                                pass
                        else:
                            continue

                        try:
                            finaldf=pd.concat([finaldf,ets_out], ignore_index=True, axis=1)
                            columnhead.append(master)
                        except:
                            continue
                        model[master]=['ETS']
                        mape[master]=[ets_mape]

                    finaldf.columns=columnhead
                    mape=mape.transpose()
                    model=model.transpose()
                    mape.columns=['Mape']
                    model.columns=['Model']
                    exportdf=finaldf.transpose()
                    exportdf["Model"]=model.Model
                    exportdf['Mape']=mape.Mape
                    exportdf.to_excel("forecastsheet.xlsx")

                    return send_file('forecastsheet.xlsx', attachment_filename='forecastsheet.xlsx')
                
                
                
        elif request.form['action'] == 'View':
            brand=request.form.getlist('brand')
            size=request.form.getlist('size')
            varietal=request.form.getlist('varietal')
            mastercodelist=request.form.getlist('mastercode')
            print(brand)
            print(size)
            print(varietal)
            print(mastercodelist)
            if len(mastercodelist)<1:
                selecteddata=masterdata[masterdata['Brand Description'].isin(brand) & masterdata['Size Description'].isin(size) & masterdata['Varietal/ Flavor Description'].isin(varietal)]
                codes=list(selecteddata['Cat Cd10 Description'].unique())
                data=pd.read_excel(r"privatedata\forecastsheet.xlsx")
                graphdata=data[data.iloc[:,0].isin(codes)]
                graphvalues=graphdata.iloc[:,1:-2].sum()
                labels=list(pd.to_datetime(graphvalues.index).map(lambda x: x.strftime("%b %y")))
                values=list(graphvalues.values)
                legend="Monthly Shipments"
                return render_template('index.html',values=values,labels=labels,legend=legend)
            else:
                data=pd.read_excel(r"privatedata\forecastsheet.xlsx")
                graphdata=data[data.iloc[:,0].isin(mastercodelist)]
                graphvalues=graphdata.iloc[:,1:-2].sum()
                labels=list(pd.to_datetime(graphvalues.index).map(lambda x: x.strftime("%b %y")))
                values=list(graphvalues.values)
                legend="Monthly Shipments"
                return render_template('index.html',values=values,labels=labels,legend=legend)
                
                

            
if __name__ == '__main__':
    app.run(port=9000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:9000/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [15/Aug/2019 11:02:58] "GET / HTTP/1.1" 200 -


csv
bestfit
size
2015-01-01
2019-07-01
6
InputFile.xlsx


C:\Users\ankit.joshi\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
C:\Users\ankit.joshi\AppData\Local\Continuum\anaconda3\lib\site-packages\pystan\misc.py:399: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonalit

Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=339.339, BIC=341.428, Fit time=0.008 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=326.443, BIC=330.621, Fit time=0.433 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=333.940, BIC=337.073, Fit time=0.046 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(2, 1, 0, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(2, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=334.906, BIC=338.039, Fit time=0.223 seconds
Fit ARIMA: order=(2, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=314.254, BIC=319.477, Fit time=0.36

...

500ML


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
C:\Users\ankit.joshi\AppData\Local\Continuum\anaconda3\lib\site-packages\pystan\misc.py:399: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
C:\Users\ankit.joshi\AppData\Local\Continuum\anaconda3\lib\site-packages\pystan\misc.py:399: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.



Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=506.214, BIC=508.303, Fit time=0.010 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=510.285, BIC=514.463, Fit time=0.056 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=508.129, BIC=511.263, Fit time=0.038 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(1, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=508.372, BIC=511.505, Fit time=0.024 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=513.104, BIC=516.237, Fit time=0.047 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=515.787, BIC=519.965, Fit time=0.101 seconds
Total fit time: 0.283 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 

C:\Users\ankit.joshi\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:165: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\ankit.joshi\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:165: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\ankit.joshi\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

INFO:werkzeug:127.0.0.1 - - [15/Aug/2019 11:05:10] "POST / HTTP/1.1" 200 -


750ML
